<a href="https://colab.research.google.com/github/faikozcan/MLFraud/blob/main/MLFraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import pandas as pd
import requests
import io

url="https://raw.githubusercontent.com/faikozcan/MLFraud/main/dataFiles/Fraud_Data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')),sep = ",",usecols=["class","device_id","source","browser","purchase_value","sex","age"])
c.head()

,purchase_value,device_id,source,browser,sex,age,class
0,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,0
1,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,0
2,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,1
3,44,ATGTXKYKUDUQN,SEO,Safari,M,41,0
4,39,NAUITBZFJKHWW,Ads,Safari,M,45,0


In [60]:
c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   purchase_value  151112 non-null  int64 
 1   device_id       151112 non-null  object
 2   source          151112 non-null  object
 3   browser         151112 non-null  object
 4   sex             151112 non-null  object
 5   age             151112 non-null  int64 
 6   class           151112 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 8.1+ MB


In [61]:
c.describe()

,purchase_value,age,class
count,151112.000000,151112.000000,151112.000000
mean,36.935372,33.140704,0.093646
std,18.322762,8.617733,0.291336
min,9.000000,18.000000,0.000000
25%,22.000000,27.000000,0.000000
50%,35.000000,33.000000,0.000000
75%,49.000000,39.000000,0.000000
max,154.000000,76.000000,1.000000
